# CCS-AMP for Unsourced Multiple Access

This notebook contains the CCS-AMP encoder/decoder for the unsourced multiple access channel using Hadamard design matrices.

The proposed algorithm goes back and forth between the inner AMP algorithm and the outer tree code. Specifically, the denoiser for the AMP is based on carrying belief propagation on the factor graph associated with the tree code, initiating the process with the updated effective observation every time.

The code is based on the following articles:
 * A Coded Compressed Sensing Scheme for Uncoordinated Multiple Access, available @ https://arxiv.org/pdf/1809.04745.pdf
 * SPARCs for Unsourced Random Access, available @ https://arxiv.org/abs/1901.06234
 * On Approximate Message Passing for Unsourced Access with Coded Compressed Sensing, available @ https://arxiv.org/pdf/2001.03705.pdf


In [1]:
import numpy as np
# import ipdb
import matplotlib.pyplot as plt

In [2]:
def fht(u):
    """
    Perform fast Hadamard transform of u, in-place.
    Note len(u) must be a power of two.
    """
    N = len(u)
    i = N>>1
    while i:
        for j in range(N):
            if (i&j) == 0:
                temp = u[j]
                u[j] += u[i|j]
                u[i|j] = temp - u[i|j]
        i>>= 1

def sub_fht(n, m, seed=0, ordering=None, new_embedding=False):
    """
    Returns functions to compute the sub-sampled Walsh-Hadamard transform,
    i.e., operating with a wide rectangular matrix of random +/-1 entries.

    n: number of rows
    m: number of columns

    It is most efficient (but not required) for max(m,n+1) to be a power of 2.

    seed: determines choice of random matrix
    ordering: optional n-long array of row indices in [1, max(m,n)] to
              implement subsampling; generated by seed if not specified,
              but may be given to speed up subsequent runs on the same matrix.

    Returns (Ax, Ay, ordering):
        Ax(x): computes A.x (of length n), with x having length m
        Ay(y): computes A'.y (of length m), with y having length n
        ordering: the ordering in use, which may have been generated from seed
    """
    assert n > 0, "n must be positive"
    assert m > 0, "m must be positive"
    if new_embedding:
        w = 2**int(np.ceil(np.log2(max(m+1, n+1))))
    else:
        w = 2**int(np.ceil(np.log2(max(m, n+1))))

    if ordering is not None:
        assert ordering.shape == (n,)
    else:
        rng = np.random.RandomState(seed)
        idxs = np.arange(1, w, dtype=np.uint32)
        rng.shuffle(idxs)
        ordering = idxs[:n]

    def Ax(x):
        assert x.size == m, "x must be m long"
        y = np.zeros(w)
        if new_embedding:
            y[w-m:] = x.reshape(m)
        else:
            y[:m] = x.reshape(m)
        fht(y)
        return y[ordering]

    def Ay(y):
        assert y.size == n, "input must be n long"
        x = np.zeros(w)
        x[ordering] = y.reshape(n)
        fht(x)
        if new_embedding:
            return x[w-m:]
        else:
            return x[:m]

    return Ax, Ay, ordering

def block_sub_fht(n, m, l, seed=0, ordering=None, new_embedding=False):
    """
    As `sub_fht`, but computes in `l` blocks of size `n` by `m`, potentially
    offering substantial speed improvements.

    n: number of rows
    m: number of columns per block
    l: number of blocks

    It is most efficient (though not required) when max(m,n+1) is a power of 2.

    seed: determines choice of random matrix
    ordering: optional (l, n) shaped array of row indices in [1, max(m, n)] to
              implement subsampling; generated by seed if not specified, but
              may be given to speed up subsequent runs on the same matrix.

    Returns (Ax, Ay, ordering):
        Ax(x): computes A.x (of length n), with x having length l*m
        Ay(y): computes A'.y (of length l*m), with y having length n
        ordering: the ordering in use, which may have been generated from seed
    """
    assert n > 0, "n must be positive"
    assert m > 0, "m must be positive"
    assert l > 0, "l must be positive"

    if ordering is not None:
        assert ordering.shape == (l, n)
    else:
        if new_embedding:
            w = 2**int(np.ceil(np.log2(max(m+1, n+1))))
        else:
            w = 2**int(np.ceil(np.log2(max(m, n+1))))
        rng = np.random.RandomState(seed)
        ordering = np.empty((l, n), dtype=np.uint32)
        idxs = np.arange(1, w, dtype=np.uint32)
        for ll in range(l):
            rng.shuffle(idxs)
            ordering[ll] = idxs[:n]

    def Ax(x):
        assert x.size == l*m
        out = np.zeros(n)
        for ll in range(l):
            ax, ay, _ = sub_fht(n, m, ordering=ordering[ll],
                                new_embedding=new_embedding)
            out += ax(x[ll*m:(ll+1)*m])
        return out

    def Ay(y):
        assert y.size == n
        out = np.empty(l*m)
        for ll in range(l):
            ax, ay, _ = sub_fht(n, m, ordering=ordering[ll],
                                new_embedding=new_embedding)
            out[ll*m:(ll+1)*m] = ay(y)
        return out

    return Ax, Ay, ordering

## Fast Hadamard Transforms

This code can all be found in `pyfht`, which uses a C extension to speed up the fht function. To make this notebook self contained, it's reproduced entirely in Python here, which will be quite slow!

Skip to the next section if you're not interested in the specific transform implementation.

In [3]:
from pyfht import block_sub_fht

# Outer Tree encoder

This function encodes the payloads corresponding to users into codewords from the specified tree code. 

Parity bits in section $i$ are generated based on the information sections $i$ is connected to

Computations are done within the ring of integers modulo length of the section to enable FFT-based BP on the outer graph

This function outputs the sparse representation of encoded messages

In [4]:
def Tree_encode(tx_message,K,messageBlocks,G,L,J):
    encoded_tx_message = np.zeros((K,L),dtype=int)
    
    encoded_tx_message[:,0] = tx_message[:,0:J].dot(2**np.arange(J)[::-1])
    for i in range(1,L):
        if messageBlocks[i]:
            # copy the message if i is an information section
            encoded_tx_message[:,i] = tx_message[:,np.sum(messageBlocks[:i])*J:(np.sum(messageBlocks[:i])+1)*J].dot(2**np.arange(J)[::-1])
        else:
            # compute the parity if i is a parity section
            indices = np.where(G[i])[0]
            ParityInteger=np.zeros((K,1),dtype='int')
            for j in indices:
                ParityInteger1 = encoded_tx_message[:,j].reshape(-1,1)
                ParityInteger = np.mod(ParityInteger+ParityInteger1,2**J)
            encoded_tx_message[:,i] = ParityInteger.reshape(-1)
    
    return encoded_tx_message



This function converts message indices into $L$-sparse vectors of length $L 2^J$.

In [5]:
def convert_indices_to_sparse(encoded_tx_message_indices,L,J,K):
    encoded_tx_message_sparse=np.zeros((L*2**J,1),dtype=int)
    for i in range(L):
        A = encoded_tx_message_indices[:,i]
        B = A.reshape([-1,1])
        np.add.at(encoded_tx_message_sparse, i*2**J+B, 1)        
    return encoded_tx_message_sparse

This function returns the index representation corresponding to a SPARC-like vector.

In [6]:
def convert_sparse_to_indices(cs_decoded_tx_message_sparse,L,J,listSize):
    cs_decoded_tx_message = np.zeros((listSize,L),dtype=int)
    for i in range(L):
        A = cs_decoded_tx_message_sparse[i*2**J:(i+1)*2**J]
        idx = (A.reshape(2**J,)).argsort()[np.arange(2**J-listSize)]
        B = np.setdiff1d(np.arange(2**J),idx)
        cs_decoded_tx_message[:,i] = B 

    return cs_decoded_tx_message

Extract information bits from retained paths in the tree.

In [7]:
def extract_msg_indices(Paths,cs_decoded_tx_message, L,J):
    msg_bits = np.empty(shape=(0,0))
    L1 = Paths.shape[0]
    for i in range(L1):
        msg_bit=np.empty(shape=(0,0))
        path = Paths[i].reshape(1,-1)
        for j in range(path.shape[1]):
            msg_bit = np.hstack((msg_bit,cs_decoded_tx_message[path[0,j],j].reshape(1,-1))) if msg_bit.size else cs_decoded_tx_message[path[0,j],j]
            msg_bit=msg_bit.reshape(1,-1)
        msg_bits = np.vstack((msg_bits,msg_bit)) if msg_bits.size else msg_bit           
    return msg_bits


## SPARC Codebook

We use the `block_sub_fht` which computes the equivalent of $A.\beta$ by using $L$ separate $M\times M$ Hadamard matrices. However we want each entry to be divided by $\sqrt{n}$ to get the right variance, and we need to do a reshape on the output to get column vectors, so we'll wrap those operations here.

Returns two functions `Ab` and `Az` which compute $A\cdot B$ and $z^T\cdot A$ respectively.

In [8]:
def sparc_codebook(L, M, n,P):
    Ax, Ay, _ = block_sub_fht(n, M, L, ordering=None)
    def Ab(b):
        return Ax(b).reshape(-1, 1)/ np.sqrt(n)
    def Az(z):
        return Ay(z).reshape(-1, 1)/ np.sqrt(n) 
    return Ab, Az

# BP on outer graph

This function computes the priors on the unknown sparse vector, given effective obervations of the (graph) neighboring sections

In [9]:
def computePrior(s,G,messageBlocks,L,M,p0,K,τ,Phat,numBPiter):
    
    q = np.zeros(s.shape,dtype=float)
    p1 = p0*np.ones(s.shape,dtype=float)
    
    for iter in range(numBPiter):
        
#         Translate the effective observation into PME. For the first iteration of BP, use the uninformative prior p0
        temp_beta = (p1*np.exp(-(s-np.sqrt(Phat))**2/(2*τ**2)))/ (p1*np.exp(-(s-np.sqrt(Phat))**2/(2*τ**2)) + (1-p1)*np.exp(-s**2/(2*τ**2))).astype(float).reshape(-1, 1)
#         temp_beta = (np.exp(-(s-np.sqrt(Phat))**2/(2*τ**2)))/ (K*np.exp(-(s-np.sqrt(Phat))**2/(2*τ**2)) + (M-K)*np.exp(Phat/(τ**2))*np.exp(-(s**2)/(2*τ**2))).astype(float).reshape(-1, 1)
        #Likelihoods = np.exp(-(s-np.sqrt(Phat))**2/(2*τ**2))/np.exp(-s**2/(2*τ**2))
        #temp_beta1 = Likelihoods/(K*Likelihoods + (M-K)*1)

        r_matrix = s.reshape(L,-1)
        lambdav_matrix = np.zeros(r_matrix.shape)
        for ell in range(L):
            r_section = r_matrix[ell]
            C0_value = (M - K) * np.exp(Phat/(τ**2))
            lambdav = (np.exp(-(r_section-np.sqrt(Phat))**2/(2*τ**2))) / (K*np.exp(-(r_section-np.sqrt(Phat))**2/(2*τ**2)) + C0_value*np.exp(-(r_section**2)/(2*(τ**2))))
            C_value = C0_value
            for step in range(40):
                if (np.sum(lambdav) < 1):
                    C_value = 0.1*C_value
                    lambdav = (np.exp(-((r_section-np.sqrt(Phat))**2 - (r_section**2))/(2*(τ**2)))) / (K*np.exp(-((r_section-np.sqrt(Phat))**2 - (r_section**2))/(2*(τ**2))) + C_value)
                else:
                    break
#             print('C_value ' + str(C_value/C0_value) + ' Final Section Sum: ' + str(np.sum(lambdav)))
#             print('Max ' +  str(np.max(lambdav)))
#             print('Min ' +  str(np.min(lambdav)))
            lambdav_matrix[ell] = lambdav

#         for i in range(L):
#             temp_beta2 = temp_beta[i*M:(i+1)*M]
#             temp_beta2 = K * temp_beta2
#             while np.sum(temp_beta2):
#                 temp_beta2 = np.sqrt(temp_beta2)
#             temp_beta2 = temp_beta2/np.sum(temp_beta2)
#             print(temp_beta2)

#         print('tau: ' + str(tau))
#         print('tau estimate: ' + str(np.sqrt(tau2_estimate)))
#         Likelihoods = np.exp(-(s-np.sqrt(Phat))**2/(2*τ**2))/np.exp(-s**2/(2*τ**2))
#         temp_beta = Likelihoods/(K*Likelihoods + (M-K)*np.exp(Phat/(τ**2)))

        #ipdb.set_trace()
        
        # Reshape PME into an LxM matrix
        Beta = temp_beta.reshape(L,-1)
        Beta = lambdav_matrix
        lambda_vector = lambdav_matrix.reshape(-1,1)
#         print('Shape Beta: ' + str(Beta.shape))
#         print('Sum Beta: ' + str(np.sum(Beta,axis=1)))
        Beta = (Beta/(np.sum(Beta,axis=1)).reshape(L,-1))
        # Rotate PME 180deg about y-axis
        Betaflipped = np.hstack((Beta[:,0].reshape(-1,1),np.flip(Beta[:,1:],axis=1)))
        # Compute and store all FFTs
        BetaFFT = np.fft.fft(Beta)
        BetaflippedFFT = np.fft.fft(Betaflipped)
        for i in range(L):
            if messageBlocks[i]:
                # Parity sections connected to info section i
                parityIndices = np.where(G[i])[0]
                BetaIFFTprime = np.empty((0,0)).astype(float)
                for j in parityIndices:
                    # Other info blocks connected to this parity block
                    messageIndices = np.setdiff1d(np.where(G[j])[0],i)
                    BetaFFTprime = np.vstack((BetaFFT[j],BetaflippedFFT[messageIndices,:]))
                    # Multiply the relevant FFTs
                    BetaFFTprime = np.prod(BetaFFTprime,axis=0)
                    # IFFT
                    BetaIFFTprime1 = np.fft.ifft(BetaFFTprime).real
                    BetaIFFTprime = np.vstack((BetaIFFTprime,BetaIFFTprime1)) if BetaIFFTprime.size else BetaIFFTprime1
                BetaIFFTprime = np.prod(BetaIFFTprime,axis=0)
            else:
                BetaIFFTprime = np.empty((0,0)).astype(float)
                # Information sections connected to this parity section (assuming no parity over parity sections)
                Indices = np.where(G[i])[0]
                # FFT
                BetaFFTprime = BetaFFT[Indices,:]
                # Multiply the relevant FFTs
                BetaFFTprime = np.prod(BetaFFTprime,axis=0)
                # IFFT
                BetaIFFTprime = np.fft.ifft(BetaFFTprime).real
            
            # Normalize to ensure it sums to one
            p1[i*M:(i+1)*M] = (BetaIFFTprime/np.sum(BetaIFFTprime)).reshape(-1,1)
            p1[i*M:(i+1)*M] = p1[i*M:(i+1)*M]/np.sum(p1[i*M:(i+1)*M])
            p1[i*M:(i+1)*M] = 1-(1-p1[i*M:(i+1)*M] )**K 
            #p1[i*M:(i+1)*M] = p1[i*M:(i+1)*M]*K/np.sum(p1[i*M:(i+1)*M])
#             print('Section ' + str(i))
#             print('Sum part ' + str(np.sum(p1[i*M:(i+1)*M])))
#             print('Max part ' + str(np.max(p1[i*M:(i+1)*M])))
#         print('Sum ' + str(np.sum(p1)))
#         print('Top ' + str(np.flip(np.sort(p1)[-5:])))
#         print('Bottom ' + str(np.flip(np.sort(p1)[:5])))
# HERE
#             print(str(np.multiply(lambdav_matrix[i].reshape(-1,1),p1[i*M:(i+1)*M]).shape))
            p1[i*M:(i+1)*M] = np.multiply(lambdav_matrix[i].reshape(-1,1),BetaIFFTprime.reshape(-1,1))
            p1[i*M:(i+1)*M] = p1[i*M:(i+1)*M]/np.sum(p1[i*M:(i+1)*M])
        #p1 = 1-(1-p1)**K 
    
    # Re-normalize priors to ensure sum is K
    #p1 = p1*K/np.sum(p1)
    print('Sum: ' + str(K*np.sum(p1)) + '; Max: ' + str(np.max(p1)))
    q=np.minimum(p1,1)
    return (K*q), lambda_vector


## AMP
This is the actual AMP algorithm. It's a mostly straightforward transcription from the relevant equations, but note we use `longdouble` types because the expentials are often too big to fit into a normal `double`.

In [10]:
def amp(y, σ_n, P, L, M, T, Ab, Az,p0,K,G,messageBlocks,BPonOuterGraph,numBPiter):

    n = y.size
    β = np.zeros((L*M, 1))
    z = y
    Phat = n*P/L
    # Store the values of τ corresponding to each iteration
    τ_evolution = np.zeros((T,1))
    
    for t in range(T):
        
        # Compute τ online using the residual
        τ = np.sqrt(np.sum(z**2)/n)
        
        # effective observation
        s = (np.sqrt(Phat)*β + Az(z)).astype(np.longdouble)
        
        if BPonOuterGraph==0:
            # Use the uninformative prior p0 for Giuseppe's scheme
            q = p0
        else:
            # Compute the prior through BP on outer graph
            q, lambda_vector = computePrior(s,G,messageBlocks,L,M,p0,K,τ,Phat,numBPiter)
            print('HERE ' + str(np.sum(q)))
        # denoiser
#         β = (q*np.exp(-(s-np.sqrt(Phat))**2/(2*τ**2)))/ (q*np.exp(-(s-np.sqrt(Phat))**2/(2*τ**2)) + (1-q)*np.exp(-s**2/(2*τ**2))).astype(float).reshape(-1, 1)
# HERE
        β = q
        print('BETA ' + str(np.sum(β)))
        # residual
#         z = y - np.sqrt(Phat)*Ab(β) + (z/(n*τ**2)) * (Phat*np.sum(β) - Phat*np.sum(β**2))
#         print((Phat*np.multiply(β,(np.ones(lambda_vector.shape) - K*lambda_vector))).shape)
#         print(((K*np.ones(β.shape))-β).shape)
        correction = np.sum(np.multiply(Phat*np.multiply(β,(np.ones(lambda_vector.shape) - K*lambda_vector)),(K*np.ones(β.shape))-β))
        z = y - np.sqrt(Phat)*Ab(β) + (z/(K*n*τ**2)) * correction
        print(correction)
        τ_evolution[t] = τ

    return β,τ_evolution

# Outer Tree decoder

This function implements the tree deocoder for a specific graph corresponding to the outer tree code

It is currently hard-coded for a specfic architecture

The architecture is based on a tri-adic design and can be found in the simulation results section of https://arxiv.org/pdf/2001.03705.pdf

In [11]:
def Tree_decoder(cs_decoded_tx_message,G,L,J,B,listSize):
    
    tree_decoded_tx_message = np.empty(shape=(0,0))
    
    Paths012 = merge_paths(cs_decoded_tx_message[:,0:3])
    
    Paths345 = merge_paths(cs_decoded_tx_message[:,3:6])
    
    Paths678 = merge_paths(cs_decoded_tx_message[:,6:9])
    
    Paths91011 = merge_paths(cs_decoded_tx_message[:,9:12])
    
    Paths01267812 = merge_pathslevel2(Paths012,Paths678,cs_decoded_tx_message[:,[0,6,12]])
    
    Paths3459101113 = merge_pathslevel2(Paths345,Paths91011,cs_decoded_tx_message[:,[3,9,13]])
    
    Paths01267812345910111314 = merge_all_paths0(Paths01267812,Paths3459101113,cs_decoded_tx_message[:,[1,4,10,14]])
    
    Paths = merge_all_paths_final(Paths01267812345910111314,cs_decoded_tx_message[:,[7,10,15]])
    
    
   
    return Paths

def merge_paths(A):
    listSize = A.shape[0]
    B = np.array([np.mod(A[:,0] + a,2**16) for a in A[:,1]]).flatten()
     
    Paths=np.empty((0,0))
    
    for i in range(listSize):
        I = np.where(B==A[i,2])[0].reshape(-1,1)
        if I.size:
            I1 = np.hstack([np.mod(I,listSize).reshape(-1,1),np.floor(I/listSize).reshape(-1,1)]).astype(int)
            Paths = np.vstack((Paths,np.hstack([I1,np.repeat(i,I.shape[0]).reshape(-1,1)]))) if Paths.size else np.hstack([I1,np.repeat(i,I.shape[0]).reshape(-1,1)])
    
    return Paths

def merge_pathslevel2(Paths012,Paths678,A):
    listSize = A.shape[0]
    Paths0 = Paths012[:,0]
    Paths6 = Paths678[:,0]
    B = np.array([np.mod(A[Paths0,0] + a,2**16) for a in A[Paths6,1]]).flatten()
    
    Paths=np.empty((0,0))
    
    for i in range(listSize):
        I = np.where(B==A[i,2])[0].reshape(-1,1)
        if I.size:
            I1 = np.hstack([np.mod(I,Paths0.shape[0]).reshape(-1,1),np.floor(I/Paths0.shape[0]).reshape(-1,1)]).astype(int)
            PPaths = np.hstack((Paths012[I1[:,0]].reshape(-1,3),Paths678[I1[:,1]].reshape(-1,3),np.repeat(i,I1.shape[0]).reshape(-1,1)))
            Paths = np.vstack((Paths,PPaths)) if Paths.size else PPaths
               
    return Paths


def merge_all_paths0(Paths01267812,Paths3459101113,A):
    listSize = A.shape[0]
    Paths1 = Paths01267812[:,1]
    Paths4 = Paths3459101113[:,1]
    Paths10 = Paths3459101113[:,4]
    Aa = np.mod(A[Paths4,1]+A[Paths10,2],2**16)
    B = np.array([np.mod(A[Paths1,0] + a,2**16) for a in Aa]).flatten()
    
    Paths=np.empty((0,0))
    
    for i in range(listSize):
        I = np.where(B==A[i,3])[0].reshape(-1,1)
        if I.size:
            I1 = np.hstack([np.mod(I,Paths1.shape[0]).reshape(-1,1),np.floor(I/Paths1.shape[0]).reshape(-1,1)]).astype(int)
            PPaths = np.hstack((Paths01267812[I1[:,0]].reshape(-1,7),Paths3459101113[I1[:,1]].reshape(-1,7),np.repeat(i,I1.shape[0]).reshape(-1,1)))
            Paths = np.vstack((Paths,PPaths)) if Paths.size else PPaths
    
    return Paths

def merge_all_paths_final(Paths01267812345910111314,A):
    
    listSize = A.shape[0]
    Paths7 = Paths01267812345910111314[:,4]
    Paths10 = Paths01267812345910111314[:,11]
    B = np.mod(A[Paths7,0] + A[Paths10,1] ,2**16)
    
    Paths=np.empty((0,0))
    
    for i in range(listSize):
        I = np.where(B==A[i,2])[0].reshape(-1,1)
        if I.size:
            PPaths = np.hstack((Paths01267812345910111314[I].reshape(-1,15),np.repeat(i,I.shape[0]).reshape(-1,1)))
            Paths = np.vstack((Paths,PPaths)) if Paths.size else PPaths
    return Paths


If tree decoder outputs more than $K$ valid paths, retain $K-\delta$ of them based on their LLRs

$\delta$ is currently set to zero

In [12]:
def pick_topKminusdelta_paths(Paths, cs_decoded_tx_message, β, J,K,delta):
    
    L1 = Paths.shape[0]
    LogL = np.zeros((L1,1))
    for i in range(L1):
        msg_bit=np.empty(shape=(0,0))
        path = Paths[i].reshape(1,-1)
        for j in range(path.shape[1]):
            msg_bit = np.hstack((msg_bit,j*(2**J)+cs_decoded_tx_message[path[0,j],j].reshape(1,-1))) if msg_bit.size else j*(2**J)+cs_decoded_tx_message[path[0,j],j]
            msg_bit=msg_bit.reshape(1,-1)
        LogL[i] = np.sum(np.log(β[msg_bit])) 
    Indices =  LogL.reshape(1,-1).argsort()[0,-(K-delta):]
    Paths = Paths[Indices,:].reshape(((K-delta),-1))
    
    return Paths


# Simulation

In [15]:
K=25 # Number of active users
B=128 # Payload size of each active user
L=16 # Number of sections/sub-blocks
n=38400 # Total number of channel uses (real d.o.f)
# T=6 # Number of AMP iterations
T=8 # Number of AMP iterations
listSize = 2*K  # List size retained for each section after AMP converges
J=16  # Length of each coded sub-block
M=2**J # Length of each section
messageBlocks = np.array([1,1,0,1,1,0,1,1,0,1,1,0,0,0,0,0]).astype(int) # Indicates the indices of information blocks
# Adjacency matrix of the outer code/graph
G = np.zeros((L,L)).astype(int)
# G contains info on what parity blocks a message is attached to and what message blocks a parity is involved with
# Currently, we do not allow parity over parities. BP code needs to be modified a little to accomodate parity over parities
G[0,[2,12]]=1
G[1,[2,14]]=1
G[2,[0,1]]=1
G[3,[5,13]]=1
G[4,[5,14]]=1
G[5,[3,4]]=1
G[6,[8,12]]=1
G[7,[8,15]]=1
G[8,[6,7]]=1
G[9,[11,13]]=1
G[10,[11,14,15]]=1
G[11,[9,10]]=1
G[12,[0,6]]=1
G[13,[3,9]]=1
G[14,[1,4,10]]=1
G[15,[7,10]]=1
BPonOuterGraph = 1 # Indicates if BP is allowed on the outer code.Setting this to zero defaults back to Giuseppe's scheme that uses uninformative prior
numBPiter = 2; # Number of BP iterations on outer code. 1 seems to be good enough
EbNodB = 2.8 # Energy per bit. With iterative extension, operating EbN0 falls to 2.05 dB for 25 users with 1 round SIC
p0 = 1-(1-1/M)**K # Giuseppe's uninformative prior
p0 = K/M# Giuseppe's uninformative prior
delta = 0
maxSims=10 # number of simulations

# EbN0 in linear scale
EbNo = 10**(EbNodB/10)
P = 2*B*EbNo/n
σ_n = 1
#Generate the power allocation and set of tau coefficients

# We assume equal power allocation for all the sections. Code has to be modified a little to accomodate non-uniform power allocations
Phat = n*P/L

msgDetected=0

for sims in range(maxSims):
    
    # Generate active users message sequences
    tx_message = np.random.randint(2, size=(K,B))
    
    # Outer-encode the message sequences
    encoded_tx_message_indices = Tree_encode(tx_message,K,messageBlocks,G,L,J)

    # Convert indices to sparse representation
    β_0 = convert_indices_to_sparse(encoded_tx_message_indices,L,J,K)
    
    # Generate the binned SPARC codebook
    Ab, Az = sparc_codebook(L, M, n, P)
    
    # Generate our transmitted signal X
    x = np.sqrt(Phat)*Ab(β_0)
    
    # Generate random channel noise and thus also received signal y
    z = np.random.randn(n, 1) * σ_n
    y = (x + z).reshape(-1, 1)

    # Run AMP decoding
    β, τ_evolution = amp(y, σ_n, P, L, M, T, Ab, Az,p0,K,G,messageBlocks,BPonOuterGraph,numBPiter)

    # Convert decoded sparse vector into vector of indices  
    cs_decoded_tx_message = convert_sparse_to_indices(β,L,J,listSize)

    # Tree decoder to decode individual messages from lists output by AMP
    Paths = Tree_decoder(cs_decoded_tx_message,G,L,J,B,listSize)
    
    # Re-align paths to the correct order
    perm = np.argsort(np.array([0,1,2,6,7,8,12,3,4,5,9,10,11,13,14,15]))
    Paths = Paths[:,perm]
    
   
    # If tree deocder outputs more than K valid paths, retain only K of them
    if Paths.shape[0] > K:
        Paths = pick_topKminusdelta_paths(Paths, cs_decoded_tx_message, β, J, K,0)

    # Extract the message indices from valid paths in the tree    
    Tree_decoded_indices = extract_msg_indices(Paths,cs_decoded_tx_message, L,J)

    # Calculation of per-user prob err
    for i in range(K):
        msgDetected = msgDetected + np.equal(encoded_tx_message_indices[i,:],Tree_decoded_indices).all(axis=1).any()

    
errorRate= (K*maxSims - msgDetected)/(K*maxSims)

print("Per user probability of error = ", errorRate)


Sum: 399.9999999999999; Max: 0.13595702000277055
HERE 400.0000000000002
BETA 400.0000000000002
26112.265358388246
Sum: 399.99999999999994; Max: 0.15612920104617328
HERE 400.0000000000002
BETA 400.0000000000002
14010.634014942847
Sum: 400.00000000000017; Max: 0.1402913711185324
HERE 400.0000000000001
BETA 400.0000000000001
14262.424933111013
Sum: 400.0; Max: 0.12662619061922548
HERE 400.00000000000006
BETA 400.00000000000006
13452.242849595945
Sum: 399.9999999999999; Max: 0.12765886314017297
HERE 400.00000000000006
BETA 400.00000000000006
13179.302845902499
Sum: 399.9999999999999; Max: 0.13020147102050073
HERE 399.99999999999966
BETA 399.99999999999966
13727.38947905549
Sum: 399.9999999999999; Max: 0.13465906706205533
HERE 400.0000000000001
BETA 400.0000000000001
13058.71169041544
Sum: 400.0000000000001; Max: 0.12992117936744516
HERE 400.0000000000002
BETA 400.0000000000002
13073.841042113525
Sum: 399.9999999999999; Max: 0.15902052329774863
HERE 400.0
BETA 400.0
27416.13543343279
Sum: 4

Sum: 399.99999999999983; Max: 0.154260792318276
HERE 400.0000000000001
BETA 400.0000000000001
11844.276743701324
Sum: 399.9999999999999; Max: 0.1649014380975237
HERE 399.9999999999998
BETA 399.9999999999998
11284.034007336957
Sum: 399.9999999999999; Max: 0.15016315011436482
HERE 400.0
BETA 400.0
14230.782410644315
Sum: 399.9999999999999; Max: 0.15676110950034522
HERE 399.9999999999999
BETA 399.9999999999999
12068.74771333163
Per user probability of error =  0.092
